<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sklearn Tuner for CNN Activity Classifier - V2

In this notebook, we use SKlearn  to tune a CNN neural net to classify PE activity.

#### Load dependencies

In [1]:
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import AveragePooling1D, LeakyReLU , MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import roc_auc_score, roc_curve 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import tensorflow as tf
print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tensorflow.__version__

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5830697819915717075
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14026761192917927219
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4022219571
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9042929706644495121
physical_device_desc: "device: 0, name: Quadro P2000, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3859967937030855165
physical_device_desc: "device: XLA_GPU device"
]
# GPUs Available:  1


'2.3.1'

#### Set hyperparameters

In [2]:
# output directory name:
output_dir = 'model_output/tunecnn-sklearn-2'
input_dir =  'Z:/Research/dfuller/Walkabilly/studies/smarphone_accel/data/Ethica_Jaeger_Merged/pocket/'
input_file_name = 'pocket-NN-data.npz'

# from the data preparation section we have:
window_size_second = 3
frequency = 30
lenght_of_each_seq = window_size_second * frequency


In [3]:
# sklearn hyperparams
params = {
    'n_conv_1':[512, 768], # filters, a.k.a. kernels
    'k_conv_1':[2, 3], # kernel length
    'n_conv_2':[256, 512], # filters, a.k.a. kernels
    'k_conv_2':[2, 3], # kernel length
    'n_conv_3':[256, 512], # filters, a.k.a. kernels
    'k_conv_3':[2, 3], # kernel length
    'maxpooling_pool_size':[3],
    'avepooling_pool_size':[3],
    'n_dense_1':[256, 512],
    'dropout_1':[0.2, 0.3],
    'n_dense_2':[256, 512],
    'dropout_2':[0.25],
    'activation_conv':['elu', 'relu', LeakyReLU()],
    'activation_dense':['elu', 'relu', LeakyReLU()]
}

# training:
n_tune_iter = 30
cv = 3
epochs = 60
batch_size = 256


#### Load data

##### For this notebook we use the acceleration data gathered from the pocket location. It was prepared in the DataPrep-Deep notebook

In [4]:
# read the raw file and get the keys:
raw_data = np.load(file=input_dir+input_file_name,allow_pickle=True)
for k in raw_data.keys():
    print(k)

acceleration_data
metadata
labels


In [5]:
# import the data

accel_array = raw_data['acceleration_data']
meta_array = raw_data['metadata']
labels_array = raw_data['labels']
input_shape = list(accel_array.shape)


#### Preprocess data

#### Convert the  labels to integer.
In the raw data format of the labels is String and there are 6 classes. 'Lying', 'Sitting', 'Self Pace walk', 'Running 3 METs',
       'Running 5 METs', 'Running 7 METs' <br>




In [6]:
n_class = len(np.unique(labels_array))
class_list, labels_array_int = np.unique(labels_array,return_inverse=True)

In [7]:
y = to_categorical(labels_array_int, num_classes=n_class)


### Splitting and shuffeling the data

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
     accel_array, y, test_size=0.1, random_state=65)


#### Design neural network architecture

In [9]:
params

{'n_conv_1': [512, 768],
 'k_conv_1': [2, 3],
 'n_conv_2': [256, 512],
 'k_conv_2': [2, 3],
 'n_conv_3': [256, 512],
 'k_conv_3': [2, 3],
 'maxpooling_pool_size': [3],
 'avepooling_pool_size': [3],
 'n_dense_1': [256, 512],
 'dropout_1': [0.2, 0.3],
 'n_dense_2': [256, 512],
 'dropout_2': [0.25],
 'activation_conv': ['elu',
  'relu',
 'activation_dense': ['elu',
  'relu',
  <tensorflow.python.keras.layers.advanced_activations.LeakyReLU at 0x1fab97d27c8>]}

In [10]:
def create_model(n_conv_1=256, k_conv_1=3, n_conv_2=256, k_conv_2=3, n_conv_3=256, k_conv_3=3,
                 maxpooling_pool_size = 2, avepooling_pool_size = 2, n_dense_1=256, dropout_1=0.2,
                 n_dense_2=256, dropout_2=0.2, activation_conv= 'relu', activation_dense= 'elu'
                ):
    model = Sequential()
    model.add(Conv1D(n_conv_1, k_conv_1, activation=activation_conv, input_shape=input_shape[1:]))
    model.add(MaxPool1D(pool_size = maxpooling_pool_size))
    model.add(Conv1D(n_conv_2, k_conv_2, activation=activation_conv))
    model.add(AveragePooling1D(pool_size = avepooling_pool_size))
    model.add(Conv1D(n_conv_3, k_conv_3, activation=activation_conv))
    # model.add(GlobalMaxPooling1D())
    model.add(GlobalAveragePooling1D())
    model.add(Dense(n_dense_1, activation=activation_dense))
    model.add(Dropout(dropout_1))
    model.add(Dense(n_dense_2, activation=activation_dense))
    model.add(Dropout(dropout_2))
    model.add(Dense(n_class, activation='softmax'))
#     model.summary()
    print(f"""n_conv_1 = {n_conv_1}, k_conv_1 = {k_conv_1},  n_conv_2 = {n_conv_2}, k_conv_2 = {k_conv_2}, 
          n_conv_3 = {n_conv_3},  k_conv_3 = {k_conv_3},  maxpooling_pool_size = {maxpooling_pool_size},
          avepooling_pool_size = {avepooling_pool_size},  n_dense_1 = {n_dense_1}, dropout_1 = {dropout_1},
          n_dense_2 = {n_dense_2}, dropout_2 = {dropout_2}, activation_conv=  {activation_conv},
          activation_dense=  {activation_dense}""")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [11]:
model_default = create_model()
model_default.summary()

n_conv_1 = 256, k_conv_1 = 3,  n_conv_2 = 256, k_conv_2 = 3, 
          n_conv_3 = 256,  k_conv_3 = 3,  maxpooling_pool_size = 2,
          avepooling_pool_size = 2,  n_dense_1 = 256, dropout_1 = 0.2,
          n_dense_2 = 256, dropout_2 = 0.2, activation_conv=  relu,
          activation_dense=  elu
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 88, 256)           2560      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 44, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 42, 256)           196864    
_________________________________________________________________
average_pooling1d (AveragePo (None, 21, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D) 

In [12]:
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size)

In [ ]:
%%time
rscv = RandomizedSearchCV(model, param_distributions=params, cv=cv, n_iter=n_tune_iter)
rscv_results = rscv.fit(X_train,y_train)

n_conv_1 = 768, k_conv_1 = 2,  n_conv_2 = 256, k_conv_2 = 3, 
          n_conv_3 = 512,  k_conv_3 = 3,  maxpooling_pool_size = 3,
          avepooling_pool_size = 3,  n_dense_1 = 512, dropout_1 = 0.3,
          n_dense_2 = 256, dropout_2 = 0.25, activation_conv=  elu,
          activation_dense=  <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000001FAC0EED048>
Epoch 1/60
152/152 [==============================] - 6s 41ms/step - loss: 1.1788 - accuracy: 0.5048
Epoch 2/60
152/152 [==============================] - 6s 38ms/step - loss: 1.0486 - accuracy: 0.5526
Epoch 3/60
152/152 [==============================] - 6s 40ms/step - loss: 0.9630 - accuracy: 0.5897
Epoch 4/60
152/152 [==============================] - 6s 39ms/step - loss: 0.8955 - accuracy: 0.6246
Epoch 5/60
152/152 [==============================] - 6s 38ms/step - loss: 0.8301 - accuracy: 0.6529
Epoch 6/60
152/152 [==============================] - 6s 38ms/step - loss: 0.7811 - accuracy: 0.6744
Epo

In [ ]:
print('Best score is: {} using {}'.format(rscv_results.best_score_,
rscv_results.best_params_))

In [ ]:
best_model = rscv_results.best_estimator_
best_param = rscv_results.best_params_

### Test the best model based on the validation data

In [ ]:
y_hat = best_model.predict(X_valid)

In [ ]:
y_hat = to_categorical(y_hat)
y_hat[0]

In [ ]:
y_valid[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
from sklearn.metrics import  accuracy_score as score
y_pred_classes = y_hat.round()
print(y_valid[0], y_hat[0], y_pred_classes[0])
acc = score(y_valid, y_pred_classes) * 100
acc

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat)*100.0

In [ ]:
"{:0.2f}".format(pct_auc)

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0:6].round(3))

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head(10)

In [ ]:
results_df = pd.DataFrame(rscv_results.cv_results_['params'])
results_df['mean'] = rscv_results.cv_results_['mean_test_score']
results_df['std'] = rscv_results.cv_results_['std_test_score']
results_df.sort_values('mean', ascending=False, ignore_index=False)

## Creating best model from the best param and train it for 60 epochs

In [ ]:
list(best_param.items())

In [ ]:
n_dense_2= 512
n_dense_1= 768
n_conv_3= 768
n_conv_2= 512
n_conv_1= 512
maxpooling_pool_size= 2
k_conv_3= 7
k_conv_2= 7
k_conv_1= 3
dropout_2= 0.25
dropout_1= 0.22
avepooling_pool_size= 2
activation= 'relu'
model = Sequential()
model.add(Conv1D(n_conv_1, k_conv_1, activation=activation, input_shape=input_shape[1:]))
model.add(MaxPool1D(pool_size = maxpooling_pool_size))
model.add(Conv1D(n_conv_2, k_conv_2, activation=activation))
model.add(AveragePooling1D(pool_size = avepooling_pool_size))
model.add(Conv1D(n_conv_3, k_conv_3, activation=activation))
# model.add(GlobalMaxPooling1D())
model.add(GlobalAveragePooling1D())
model.add(Dense(n_dense_1, activation=activation))
model.add(Dropout(dropout_1))
model.add(Dense(n_dense_2, activation=activation))
model.add(Dropout(dropout_2))
model.add(Dense(n_class, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])


In [ ]:
output_dir= 'model_ouput/tune-sklearn-1'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
modelcheckpoint = ModelCheckpoint(filepath=output_dir+
                                  "/weights.{epoch:02d}.hdf5")

In [ ]:
model.fit(X_train, y_train, 
         batch_size=batch_size, epochs=epochs, verbose=1, 
         validation_data=(X_valid, y_valid), 
         callbacks=[modelcheckpoint])

In [ ]:
# model.load_weights(output_dir+"/weights.49.hdf5") # 94.22 val accuracy

In [ ]:
y_hat = model.predict(X_valid)

In [ ]:
len(y_hat)

In [ ]:
y_hat[0]

In [ ]:
y_valid[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat)*100.0

In [ ]:
"{:0.2f}".format(pct_auc)

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0:6].round(3))

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head(10)